# Recipe Shopping List

This notebook generates a shopping list for recipe ingredient list specified in a spreadsheet saved as TSV (tab).

In [2]:
open System.IO
open System.Text.RegularExpressions

type RecipeIngredient =
    {
        Recipe : string
        Ingredient : string
        Quantity : float
        Unit : string
        Location : float
    } with 
    static member Create(recipe, ingredient, quantity, unit, location ) = { Recipe = recipe; Ingredient = ingredient; Quantity = quantity; Unit = unit; Location = location }

//matches a numeric decimal quantity followed by a non numeric unit identifier, e.g. "1.5oz"
let quantityRegex = new Regex( @"([0-9\.]+)\s*(\D+)")

//Map of recipe name to list of ingredients
let db = 
    Directory.GetFiles(".","*.tsv").[0] //will take the first file we find
    |> File.ReadAllLines
    |> Array.skip 1                     //skip header
    |> Array.map( fun row ->
        let s = row.Split('\t')
        
        //last col mixes quantity and unit
        let m = quantityRegex.Match(s.[2])
        
        //if we have quantity and unit (e.g. 1.5oz)
        if m.Groups.Count = 3 then
            { Recipe=s.[0]; Ingredient=s.[1]; Quantity=m.Groups.[1].Value|>float; Unit=m.Groups.[2].Value; Location = s.[3] |> float }
        //unit is identity (e.g. 1 carrot)
        else
            { Recipe=s.[0]; Ingredient=s.[1]; Quantity=s.[2]|>float; Unit=""; Location = s.[3] |> float}
    )
    |> Array.groupBy( fun ri -> ri.Recipe )
    |> Map.ofArray
    
//display the recipes we know about
let format ( s : string ) = s.Replace(";",";\n")
db |> Map.toArray |> Array.map fst |> sprintf "%A" |> format

"[|"baked ziti chickpeas";
 "cabbage cheddar pie";
 "celery curry";
 "chana masala";

  "cream of lentil soup";
 "cuban black beans";
 "edamame fried rice";

  "edamame noodles";
 "lentils with fruit and sweet potatoes";
 "okra curry";

  "penne all'arrabbiata";
 "portabello mushroom ragout";
 "potato chilli burro";

  "spinach enchiladas verde";
 "spinach lasagne";
 "spinach lentil soup";

  "sweet potato beetroot curry";
 "sweet potato black bean quessadillas";

  "sweet potato burritos";
 "szechuan tofu broccoli mushrooms";

  "two step southwestern stew";
 "veggie burros";
 "zucchini tostadas"|]"

**Copy the list above below and remove the recipes you don't want**
(TODO checkbox GUI)

In [3]:
let recipes =
    [|"baked ziti chickpeas";
 "celery curry";
 "chana masala";

  "cream of lentil soup";
 "cuban black beans";
 "edamame fried rice";

  "edamame noodles";
 "lentils with fruit and sweet potatoes";
 "okra curry";

  "penne all'arrabbiata";
 "portabello mushroom ragout";
 "potato chilli burro";

  "spinach enchiladas verde";
 "spinach lasagne";

  "sweet potato beetroot curry";

  "sweet potato burritos";
 "szechuan tofu broccoli mushrooms";

  "two step southwestern stew";
 "veggie burros";
 "zucchini tostadas"|]
 
//-----------------------------
// given selected recipes, collect all ingredients as groups, and sum their quantities
let ingredients =
    recipes
    
    //index by ingredient
    |> Array.collect( fun r ->
        db.[r] |> Array.map( fun ri -> ri.Ingredient,ri )
    )
    |> Array.sortBy( fun (ingredient,ri) -> ri.Location )
    |> Array.groupBy fst

    //aggregate quantities for each ingredient as much as possilbe
    |> Array.map( fun (i,tuples) -> 
    
        //sum quantities of the same unit
        let unitQuantities = 
            tuples
            |> Array.map snd
            |> Array.groupBy( fun ri2 -> ri2.Unit ) 
            |> Array.map( fun (u,ris2) ->
                (ris2 |> Array.sumBy( fun ri2 -> ri2.Quantity ) |> string) + " " + u
            )
        
        //map to string
        i + ":" + (unitQuantities |> String.concat "; ")
    )
    
ingredients |> String.concat "\n"

"cherry tomatoes:8 oz
onion:9 
broccoli:1 ; 1 c
cremini mushrooms:8 oz
okra:6 c
banana:3 
green bell pepper:2 
sweet potato:4 c; 1 large; 4 
portabello:1.5 c
garlic:27 clove
green onion:14 
lemon:2 
carrot:8 
yellow bell pepper:1 
green onions:5 
ginger:6 in; 4 tb
plantain:1 
pear:2 
spinach:42 oz
tomatoes:3 can
zucchini:8 
potatos:3 
cauliflower:1 
celery:1 
cilantro:0.75 c
chillis:2 
stewed tomatoes:1 can
green salsa:7 oz
cream of mushroom soup:36 oz
green chillis:8 oz
tortillas:43 
chickpeas:4 can
penne pasta:24 oz
kidney beans:6 c; 2 can
spaghetti sauce:1 jar
salsa:2 c
hominy:1 can
tomatoes green chillies:5 can
cooking sherry:0.25 c
soy sauce:0.5 c; 7 tb
chili paste with garlic:1 tb
beetroot:1 can
lasagne noodles:8 oz
green beans:1 can
crushed tomatoes:1 can
lentils:1.5 c; 16 oz
rice vinegar:2 tb
udon noodles:9 oz
rice:5.5 c
pineapple:1 can
tomato paste:1 tb
stock cubes:2 
chipotle adobo sauce:1 t
kalamata olives:0.25 c
tomato puree:1 c
tomatoes basil garlic oregano:1 can
diced tom